In [1]:
import os
from typing import Any

from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf
import json
# import tesseract
import pytesseract
from typing import *
from box import ConfigBox
from pathlib import Path
import re, sys, os
from functools import partial



import io
import os
import base64
import numpy as np
from PIL import Image
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, SystemMessage, AIMessage


import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever


from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

from dotenv import load_dotenv

from dataclasses import dataclass
from zipfile import ZipFile
load_dotenv()

/Users/goldyrana/miniconda3/envs/bounce/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
os.getcwd()
os.chdir("../../")

In [3]:
from unstructured.documents.elements import CompositeElement, Table, Image, ListItem  # Import ListItem

from backend.src.utils.common import *
from backend.logger import logger
from backend.src.constants import *

In [4]:



class Extractor:
    def __init__(self):
        pass
    
    def get_metadata(raw_pdf_element: List) -> List:
        """To get the metadata of elements

        Args:
            raw_pdf_element (List): list of elements in pdf

        Returns:
            List: containing meta data
        """
        metadata = []
        for element in raw_pdf_element:
            metadata.append(element.metadata.to_dict())
        
        return metadata
    
    def add_metadata(self, element, **kwargs):
        metadata = kwargs
        return Document(element, metadata)
 
    def get_year(self, file: str) -> str:
        """To get the year from file name
        Args:
            file: name of with extension"""
        
        year = re.search(r"\d{4}", file).group()    
        if year:
            return str(year)
        else:
            try:
                file_name = os.path.split(file)[-1].split(".")[0]
                return file_name
            except Exception as e:
                logger.error(f"Error in getting year from file: {file} with error: {e}")
                return "1.0"
        
    def extract_data(self, pdf_file, **kwargs):
        raw_pdf_elements = partition_pdf(filename=pdf_file, **kwargs)
        return raw_pdf_elements
    
    def generate_unique_id(self, data: Iterator)-> List[str]:
        """To generate unique id for each element"""
        
        unique_id = []
        for _ in data:
            unique_id.append(str(uuid.uuid4()))
        
        return unique_id
    
    def generate_document(self, content: List, id_key: str):
        """To generate document from content"""
                
        ids = self.generate_unique_id(content)
        summaries = [Document(page_content=summary, metadata={id_key: ids[index]})
                         for index, summary in enumerate(content)]
        
        return summaries
    
    @staticmethod
    def seprate_data_metadata_for_text(data: List[CompositeElement]) -> Tuple[List, List]:
        """To seprate data and metadata from dict

        Args:
            data (List[dict]): List containing raw_pdf_elements
            text_name (str): key to get text from dict. Defaults to "text"
            metadata_name (str): key to get metadata from dict. Defaults to "metadata"

        Returns:
            Tuple: containing text, metadata
        """
        text, metadata = [], []
        for i in data:
            text.append(str(i))
            metadata.append(i.metadata.to_dict())
        return text, metadata
        

@dataclass
class VectorDatabaseConfig:
    vectorstore_name: str
    persist_directory: str
        
class ConfigurationManager:
    def __init__(self, CONFIG_FILE_PATH, PARAMS_FILE_PATH):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        
    def get_vectordatabase_config(self) -> VectorDatabaseConfig:
        params = VectorDatabaseConfig(
            vectorstore_name=self.config.vector_database.vectorstore_name,
            persist_directory=self.config.vector_database.persist_directory)
        return params
    
    
class VectorDatabase(Extractor):
    def __init__(self, config):
        super(VectorDatabase, self).__init__()
        self.config = config
    
    def init_chromadb(self, embeddings):
        return Chroma(self.config.vectorstore_name, 
                      embeddings, 
                      persist_directory=self.config.persist_directory)
        
    def get_retriever(self, vectorstore, id_key: str="doc_id") -> MultiVectorRetriever:
        store = InMemoryStore()
        return MultiVectorRetriever(
            vectorstore=vectorstore,
            docstore=store,
            id_key=id_key
        )
          
    def generate_document(self, data, metadata=None) -> Document:
        if metadata == None:
            metadata = self.generate_unique_id(data)
        
        documents = [Document(page_content=data_, metadata=metadata_)
                        for data_, metadata_ in zip(data, metadata)]
        return documents
    
    def store_to_vb(self, data: List[Tuple], summaries: List[Document], retriever)->None:
        try:
            retriever.docstore.mset(data)
            retriever.vectorstore.add_documents(summaries)
        except Exception as e:
            print("Error in storing data to vectorbase: ", e)
            raise e

    def zip_vector_database(self) -> None:
        """To zip the vector database; using persist_directory
        """
        with ZipFile(self.config.persist_directory + ".zip", 'w') as zipf:
            for root, dirs, files in os.walk(self.config.persist_directory):
                for file in files:
                    zipf.write(os.path.join(root, file))

    def unzip_vectorbase(self, extract_to) -> None:
        """To unzip vectorbase; using persist_directory
        Args:
            extract_to: path to extract the vectorbase
        """
        with ZipFile(self.config.persist_directory + ".zip", 'r') as zip_ref:
            zip_ref.extractall(extract_to)


In [5]:
config_manager = ConfigurationManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH)
vb_params = config_manager.get_vectordatabase_config()
print(vb_params)
vb = VectorDatabase(vb_params)
vectorstore = vb.init_chromadb(OpenAIEmbeddings())

retriever = vb.get_retriever(vectorstore, id_key="doc_id")
# vb.store_to_vb(encoded_images, image_summaries, retriever)

2025-02-10 11:20:00,701 - root - INFO - Yaml read successfully from config.yaml
2025-02-10 11:20:00,702 - root - ERROR - FileNotFoundError: params.yaml
/var/folders/rt/_sv9t5px3yv5bth97z8py2mr0000gn/T/ipykernel_95853/3471854416.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  vectorstore = vb.init_chromadb(OpenAIEmbeddings())


VectorDatabaseConfig(vectorstore_name='test3', persist_directory='test_directory')


/var/folders/rt/_sv9t5px3yv5bth97z8py2mr0000gn/T/ipykernel_95853/1953380339.py:106: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  return Chroma(self.config.vectorstore_name,
2025-02-10 11:20:01,513 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-02-10 11:20:01,618 - chromadb.config - DEBUG - Starting component System
2025-02-10 11:20:01,618 - chromadb.config - DEBUG - Starting component Posthog
2025-02-10 11:20:01,619 - chromadb.config - DEBUG - Starting component OpenTelemetryClient
2025-02-10 11:20:01,619 - chromadb.config - DEBUG - Starting component SqliteDB
2025-02-10 11:20:01,6

2025-02-10 11:20:02,134 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): us.i.posthog.com:443
2025-02-10 11:20:03,170 - urllib3.connectionpool - DEBUG - https://us.i.posthog.com:443 "POST /batch/ HTTP/1.1" 200 15


In [47]:

from base64 import b64decode
from IPython.display import display, HTML

class QueryHandler:
    def __init__(self, retriever):
        self.retriever = retriever
        self.docstore = retriever.docstore
        self.vectorstore = retriever.vectorstore
        
    def similarity_search(self, query: str, top_k: int=5):
        return self.retriever.vectorstore.similarity_search(query, top_k)
    
    def relevant_documents(self, query: str, top_k: int=5):
        return self.retriever.get_relevant_documents(query, k=top_k, return_metadata=True)
    
    def query_by_vector(self, vector: np.array, top_k: int=5):
        return self.retriever.query_by_vector(vector, top_k)
    
    def query_by_document(self, document: Document, top_k: int=5):
        return self.retriever.query_by_document(document, top_k)
        
    def map_raw_docs(self, vectorstore_results: List[Document]) -> List:
        """To map the raw documents from vectorstore results by using doc_id from metadata
        Args:
            vectorstore_results (List[Document]): results from vectorstore
        Returns:
            List: containing raw documents
        """
        raw_docs = []
        meta_data = []
        for i in vectorstore_results:

            doc_id = i.metadata["doc_id"]
            doc = retriever.docstore.__dict__["store"].get(doc_id, "None") # if data is present in InMemoryStore
            if doc == "None":
                raw_docs.append(i.metadata.get("raw_text", "None"))
            else:
                raw_docs.append(doc)
            i.metadata.pop("raw_text", None)
            meta_data.append(i.metadata)
        return raw_docs, meta_data
    
    def split_image_text_types(self, docs: List[Document]) -> Dict:
    #     ''' Split base64-encoded images and texts '''
        b64 = []
        text = []
        for doc in docs:
            if doc.metadata["type"] == "Image":
                b64.append(self.query_by_id(doc.id))
            elif doc.metadata["type"] == "Text":
                text.append(doc)

        return {
            "images": b64,
            "texts": text
        }

    @staticmethod
    def plt_img_base64(img_base64):

        # Create an HTML img tag with the base64 string as the source
        image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'

        # Display the image by rendering the HTML
        display(HTML(image_html))
        
    def plot_img_base64(self, img_base64: str):
        img = Image.open(io.BytesIO(b64decode(img_base64)))
        img.show()
        
query_handler = QueryHandler(retriever)


In [48]:
reterieved_docs = query_handler.similarity_search("what does report tells us", 5)
raw_docs, metadata = query_handler.map_raw_docs(reterieved_docs)

2025-02-10 12:05:50,113 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x3357c1700>, 'json_data': {'input': [[12840, 1587, 1934, 10975, 603]], 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
2025-02-10 12:05:50,117 - openai._base_client - DEBUG - Sending HTTP Request: POST https://api.openai.com/v1/embeddings
2025-02-10 12:05:50,120 - httpcore.connection - DEBUG - close.started
2025-02-10 12:05:50,121 - httpcore.connection - DEBUG - close.complete
2025-02-10 12:05:50,122 - httpcore.connection - DEBUG - connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=None socket_options=None
2025-02-10 12:05:50,206 - httpcore.connection - DEBUG - connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x334771a00>
2025-02-10 12:05:50,209 - httpcore.connection - DEBUG - start_tls.started ssl_context=<ssl.SSLContext o

In [49]:
reterieved_docs

[Document(metadata={'doc_id': '294c2b57-670c-437c-8aee-547771c9df5e', 'file_directory': 'backend/data/raw_pdfs', 'filename': '2023_removed.pdf', 'filetype': 'application/pdf', 'languages': 'eng', 'last_modified': '2025-02-06T12:34:29', 'orig_elements': 'eJy1WFFvGzcS/iuEXq4FZJ+sJI4dPRVp79CHuwZNmj4EgUFxuVqeueSW5EpWiv73fjMkVyvb1ysOdw9BvBTJmflm5psZfvp1oa3utUt3plm8EQt5c/vy9pVeX6ibbXPxcvVifXG7pb9e3yq5Vbfy9Wu9WIpFr5NsZJI48+tCeR8a42TSkb+tPPox3XXa7LqElatXqxXOlOWDaVKH1fX19WusDt64ROc+fbq+uXy5FK9uVperz0tRPq+urq8vb+h7fb16cXn73Eo+gpVFPMake7LknXnQ9v0glV78hh8anbRKxrs7ZWWMd0PwW2xbXd6+Wq9usKE1VqfjoPnsu38sWGG3G+WOrfq00G63+MyrMd31vjGt0YzZerV+dbFaX6yuP1yt37x4+WZ9S6cHnLxzY7/VgTAgJZJ+IDwWP3c66KUwTkh3FK0PBxmaC+v9vXE7EROQJJ8sReq0UL4faJt+GIKOUUccog+YI8kg4YPYamt0K2QUyYt2TGPQAptHm+JSxFF1/+aAiWIro24EVnvpoDHJ/UsUO+8b0UqTOjEABpKJTdt/EYZ7qDC6RgdWD2CO/UD3RhF1ImtwSG79XouvRmehsnA+kQxsDwcT9dd8G6uwxzpU3pK6Mnont5auFL4V3wzBWHG1XgoA/CJjgZDT9FvqoDnBAW3ZpkvxocPXCTtoDAwgWMCFQbqkNd9awBl0gKIwWWlWxri9t9BY6ZAk/BJMvM9Gj66sJaPzCl0cR8

In [44]:
reterieved_docs[0]

Document(metadata={'doc_id': '294c2b57-670c-437c-8aee-547771c9df5e', 'file_directory': 'backend/data/raw_pdfs', 'filename': '2023_removed.pdf', 'filetype': 'application/pdf', 'languages': 'eng', 'last_modified': '2025-02-06T12:34:29', 'orig_elements': 'eJy1WFFvGzcS/iuEXq4FZJ+sJI4dPRVp79CHuwZNmj4EgUFxuVqeueSW5EpWiv73fjMkVyvb1ysOdw9BvBTJmflm5psZfvp1oa3utUt3plm8EQt5c/vy9pVeX6ibbXPxcvVifXG7pb9e3yq5Vbfy9Wu9WIpFr5NsZJI48+tCeR8a42TSkb+tPPox3XXa7LqElatXqxXOlOWDaVKH1fX19WusDt64ROc+fbq+uXy5FK9uVperz0tRPq+urq8vb+h7fb16cXn73Eo+gpVFPMake7LknXnQ9v0glV78hh8anbRKxrs7ZWWMd0PwW2xbXd6+Wq9usKE1VqfjoPnsu38sWGG3G+WOrfq00G63+MyrMd31vjGt0YzZerV+dbFaX6yuP1yt37x4+WZ9S6cHnLxzY7/VgTAgJZJ+IDwWP3c66KUwTkh3FK0PBxmaC+v9vXE7EROQJJ8sReq0UL4faJt+GIKOUUccog+YI8kg4YPYamt0K2QUyYt2TGPQAptHm+JSxFF1/+aAiWIro24EVnvpoDHJ/UsUO+8b0UqTOjEABpKJTdt/EYZ7qDC6RgdWD2CO/UD3RhF1ImtwSG79XouvRmehsnA+kQxsDwcT9dd8G6uwxzpU3pK6Mnont5auFL4V3wzBWHG1XgoA/CJjgZDT9FvqoDnBAW3ZpkvxocPXCTtoDAwgWMCFQbqkNd9awBl0gKIwWWlWxri9t9BY6ZAk/BJMvM9Gj66sJaPzCl0cR8a

In [45]:
raw_docs, meta_data = raw_docs

In [46]:
meta_data

[{'doc_id': '294c2b57-670c-437c-8aee-547771c9df5e',
  'file_directory': 'backend/data/raw_pdfs',
  'filename': '2023_removed.pdf',
  'filetype': 'application/pdf',
  'languages': 'eng',
  'last_modified': '2025-02-06T12:34:29',
  'orig_elements': 'eJy1WFFvGzcS/iuEXq4FZJ+sJI4dPRVp79CHuwZNmj4EgUFxuVqeueSW5EpWiv73fjMkVyvb1ysOdw9BvBTJmflm5psZfvp1oa3utUt3plm8EQt5c/vy9pVeX6ibbXPxcvVifXG7pb9e3yq5Vbfy9Wu9WIpFr5NsZJI48+tCeR8a42TSkb+tPPox3XXa7LqElatXqxXOlOWDaVKH1fX19WusDt64ROc+fbq+uXy5FK9uVperz0tRPq+urq8vb+h7fb16cXn73Eo+gpVFPMake7LknXnQ9v0glV78hh8anbRKxrs7ZWWMd0PwW2xbXd6+Wq9usKE1VqfjoPnsu38sWGG3G+WOrfq00G63+MyrMd31vjGt0YzZerV+dbFaX6yuP1yt37x4+WZ9S6cHnLxzY7/VgTAgJZJ+IDwWP3c66KUwTkh3FK0PBxmaC+v9vXE7EROQJJ8sReq0UL4faJt+GIKOUUccog+YI8kg4YPYamt0K2QUyYt2TGPQAptHm+JSxFF1/+aAiWIro24EVnvpoDHJ/UsUO+8b0UqTOjEABpKJTdt/EYZ7qDC6RgdWD2CO/UD3RhF1ImtwSG79XouvRmehsnA+kQxsDwcT9dd8G6uwxzpU3pK6Mnont5auFL4V3wzBWHG1XgoA/CJjgZDT9FvqoDnBAW3ZpkvxocPXCTtoDAwgWMCFQbqkNd9awBl0gKIwWWlWxri9t9BY6ZAk/BJMvM9Gj66sJaPzCl0cR8aCdFcd

# content generation

In [ ]:
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

def prompt_func(dict):
    format_texts = "\n".join(dict["context"]["texts"])
    return [
        HumanMessage(
            content=[
                {"type": "text", "text": f"""Answer the question based only on the following context, which can include text, tables, and the below image:
Question: {dict["question"]}

Text and tables:
{format_texts}
"""},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{dict['context']['images'][0]}"}},
            ]
        )
    ]

model = ChatOpenAI(temperature=0, model="gpt-4-vision-preview", max_tokens=1024)

# RAG pipeline
retriever
chain = (
    {"context": retriever | RunnableLambda(split_image_text_types), "question": RunnablePassthrough()}
    | RunnableLambda(prompt_func)
    | model
    | StrOutputParser()
)